# Quote Scraping Code

### The challenge in this portion of the project is to produce data that is properly organized and presented in a manner that faciltates the storage in a database.  While one could write simple 'find_all' code to scrape data from the site, it would not A. be automated B. be organized and C. nor structured appropriately for storage.  
### The approach taken here was to ensure that the code provided for ensuring that all of these points were considered in designing the code.  First the code needed to provide for a one touch crawl of the whole site that is both complete and rapid/efficient.  The second requirement was to maintaion the orgnaization of the data.  Simple scraping commands will return unorganized data that retains none of its value once collected as the datas relationships and organization are integral to its value.  The third achieved goal for the code was to present the data prepared for database storage.  This involved evaluating the data and determining that fundamentally the data would best be stored as 2 tables: one for quote data and the other for author data and having them joined on the authors name.  Thus the code return 2 lists: one with with a lists of lists of the quote data (Quote, Author, Tags) and the other with a list of dictionaries containing the organized Author data (Name, Birthdate, and Bio)
### Overview of the code :  After the importing the required libaries and initiating the broswser the two database prepared output lists  are defined. Next the main loop that is the basis of the automation is established that will crawl the whole site to scrape the data.  It begins on the home page and branches in a down then sideways direction.  The first main portion of the code is the main code for crawling the quote data and placing it into a list of lists.  It uses a series of loops to deal with the one big 'gotcha' that exists in the quote data that makes simple scrape code irrelevant because it will not be able to return the data appropriately. The second part of the main code is focused on the sideways  crawl of the author data.  The code opens each of the pages and scrapes the required data and places it an organized dictionary which is then appended the master list for output.  

In [ ]:
import os
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
from flask import Flask, redirect, render_template, jsonify
import time
from selenium import webdriver
import lxml
import urllib

browser=Browser("chrome", headless=False)

total_quotes=[]
author_list=[]

for r in range(1,11):
    url=f'http://quotes.toscrape.com/page/{r}'
    response=requests.get(url)
    browser.visit(url)
    soup=bs(response.text, "lxml")

    ascrape=soup.find_all('div',{'class':'quote'})
    for a in ascrape:
        tags_list=[]
        text=a.find('span', class_='text').text
        author=a.find('small', class_='author').text
        tags=a.find('div', class_='tags').find_all('a')
        for tag in tags:
            tag_text=tag.text.strip()
            tags_list.append(tag_text)
        one_quote=[text,author,tags_list]
        total_quotes.append(one_quote)

    about_links=browser.find_by_text("(about)")
    href_list=[]
    for l in about_links:
        ref=l['href']
        href_list.append(ref)
    for h in href_list:
        print(h)
        browser.visit(h)
        req=requests.get(h)
        soup=bs(req.text, "lxml")
        author_birth=soup.find('span', class_="author-born-date").text
        author_desc=soup.find('div', class_="author-description").text
        author_name=soup.find('h3', class_="author-title").text
        author_dict={'Name':author_name.strip(), 'Birthdate': author_birth.strip(), 'Bio':author_desc.strip()}
        author_list.append(author_dict)